## installing SnapMl library

##### we are going to use python API which is offered by Snap Machin Learning.
##### SnapMl is a hig performance library from IBM.
##### It provides highly efficient CPU/GPU implementation of linear model and tree based model.
##### Here we are going to use a Decision tree and Support Vector Machine models.

In [1]:
!pip install snapml


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
